In [1]:
import cv2
import os
from PIL import Image, ImageSequence
import numpy as np

In [2]:
def process_gif_image_greyscale(gif_path):
    with Image.open(gif_path) as img:
        frame = ImageSequence.Iterator(img)[0]
        frame = frame.convert('L')  # Convert to grayscale
        open_cv_image = np.array(frame) 
        return open_cv_image
    
def detect_and_save_faces(image_path, cascade, output_folder):
    # Check if the image is a GIF
    if image_path.lower().endswith('.gif'):
        image = process_gif_image_greyscale(image_path)
    else:
        # Read the image normally
        image = cv2.imread(image_path)

    # If the image is not loaded, create a white image
    if image is None:
        print(f"Failed to load image: {image_path}. Creating a white image.")
        white_image = np.ones((450, 450, 3), dtype=np.uint8) * 255
        white_image_filename = os.path.join(output_folder, f'{os.path.basename(image_path)}')
        cv2.imwrite(white_image_filename, white_image)
        return  # Skip this image and return early

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Crop and save faces
    for i, (x, y, w, h) in enumerate(faces):
        face = image[y:y+h, x:x+w]
        face_filename = os.path.join(output_folder, f'{os.path.basename(image_path)}')
        cv2.imwrite(face_filename, face)



In [3]:
# image_path = r"C:\Users\JKLiu\OneDrive\Desktop\Spring2024\ECE50024\Challenge\train_small\train_small\1.jpg"
# output_folder = r"C:\Users\JKLiu\OneDrive\Desktop\Spring2024\ECE50024\Challenge\cutted_train_small"

# cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# detect_and_save_faces(image_path= image_path, cascade= cascade, output_folder=output_folder)
def process_folder(folder_path, output_folder):
    # Load the pre-trained Haar Cascade model
    cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Ensure output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each image in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            detect_and_save_faces(os.path.join(folder_path, filename), cascade, output_folder)

In [5]:
output_folder = r"D:\Challenge\cutted_grascale"
input_folder = r"D:\Challenge\grayscale_train"

process_folder(folder_path= input_folder, output_folder= output_folder)
